In [22]:
import pickle as pickle 
import random
import operator
from itertools import permutations 

In [3]:
import numpy as np
import pandas as pd

In [ ]:
from torch# import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer

In [6]:
d = pd.read_csv('data/final_proj_data_preprocessed_1000sample.csv')

In [ ]:
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def bert_nsp(seq_A, seq_B):
    encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt')
    try: 
        seq_relationship_logits = model(**encoded)[0]
    except:
        similarity = 0
    probs = softmax(seq_relationship_logits, dim=1)
    similarity = probs.numpy()[0][0]
    return similarity

In [ ]:
def One_prob_clustering(posts, m,n):
    unselected_posts = posts.copy()
    clusters = {}
    while len(unselected_posts)! = 0:
        select_post = random.choice(unselected_posts)
        sim_dict = {}
        for p in unselected_posts:
            similarity = bert_nsp(t[select_post][0], t[p][1])
            sim_dict[p] = similarity
        sorted_sim_list = sorted(sim_dict.items(), key=operator.itemgetter(1),reverse=True)
        most_similar = sorted_sim_list[range(floor(m/n))]
        clusters[select_post] = most_similar
        for post in most_similar:
            unselected posts.remove(post)
    return clusters

In [ ]:
def Merge_multiple_prob_clustering(p, m, posts,n):
    similarity_table = {}
    for i in range(n):
        similarity_table[i] = np.zeros(n)
    for i in range(p):
        one_probabilistic_clustering = One_prob_clustering(posts, m, n)
        for j in one_probabilistic_clustering.keys():
            one_cluster =  [j] + [one_probabilistic_clustering[j]]
            all_similar_pairs = permutations(one_cluster, 2)
            for k in all_similar_pairs:
                similarity_table[k[0]][k[1]] += 1
    return similarity_table

In [ ]:
posts = d.id.to_list()
n = 1000
p =5 
m =2

In [ ]:
similarity_table = Merge_multiple_prob_clustering(p, m, posts,n)